## Deploy Modelo 

In [64]:
import joblib

# clf = joblib.load(r'D:/Github/Forked/crypto_technical_analysis/models/logistic_regression_model_bl_target_10_7d.joblib')
# clf = joblib.load(r'D:/Github/Forked/crypto_technical_analysis/models/trained_withoutSymbols/logistic_regression_model_without_symbolbl_target_10_15d.joblib')
clf = joblib.load(r'D:/Github/Forked/crypto_technical_analysis/models/trained_v1/logistic_regression_model_v1_bl_target_10_7d.joblib')

In [65]:
from pathlib import Path
import pandas as pd
import numpy as np

pd.set_option("display.max_columns", None)

input_path = r'D:\Github\Forked\crypto_technical_analysis\files\crypto_data_with_indicators.parquet'

dados = pd.read_parquet(input_path)

In [66]:
target_list_bol =   [
    # boleanos
    'bl_target_10_7d','bl_target_15_7d','bl_target_20_7d','bl_target_25_7d',
    'bl_target_10_15d','bl_target_15_15d','bl_target_20_15d','bl_target_25_15d', 
    'bl_target_10_30d','bl_target_15_30d','bl_target_20_30d','bl_target_25_30d' 
]

target_list_val =   [
    # percentual
    'target_10_7d','target_15_7d','target_20_7d','target_25_7d',
    'target_10_15d','target_15_15d','target_20_15d','target_25_15d', 
    'target_10_30d','target_15_30d','target_20_30d','target_25_30d', 
]

remove_target_list = target_list_bol + target_list_val

removing_cols = ['Date', 'Symbol', 'Dividends', 'Stock Splits']

In [67]:
last_date = dados[dados['Date'] == dados['Date'].max()]
len(last_date)

683

In [68]:
dados_x = last_date.drop(last_date[remove_target_list], axis=1)
dados_x = dados_x.drop(dados_x[removing_cols], axis=1)

len(dados_x.columns)

40

In [72]:
dummies_recent = pd.get_dummies(dados_x)
display(dummies_recent.head(2))
len(dummies_recent.columns)

,Open,High,Low,Close,Volume,ema_12,ema_26,ema_50,ema_100,ema_200,ema_12_above_ema_26,ema_12_above_ema_50,ema_12_above_ema_100,ema_12_above_ema_200,ema_26_above_ema_12,ema_26_above_ema_50,ema_26_above_ema_100,ema_26_above_ema_200,ema_50_above_ema_12,ema_50_above_ema_26,ema_50_above_ema_100,ema_50_above_ema_200,ema_100_above_ema_12,ema_100_above_ema_26,ema_100_above_ema_50,ema_100_above_ema_200,ema_200_above_ema_12,ema_200_above_ema_26,ema_200_above_ema_50,ema_200_above_ema_100,vl_adx,vl_dmp,vl_dmn,rsi,vl_macd,vl_macd_signal,vl_macd_delta,qt_days_macd_delta_positive,nm_adx_trend_1. Absent or Weak Trend,nm_adx_trend_2. Strong Trend,nm_adx_trend_3. Very Strong Trend,nm_adx_trend_4. Extremely Strong Trend,nm_rsi_trend_rsi_0_to_10,nm_rsi_trend_rsi_11_to_20,nm_rsi_trend_rsi_21_to_30,nm_rsi_trend_rsi_31_to_40,nm_rsi_trend_rsi_41_to_50,nm_rsi_trend_rsi_51_to_60,nm_rsi_trend_rsi_61_to_70,nm_rsi_trend_rsi_71_to_80,nm_rsi_trend_rsi_91_to_100
205,0.002310,0.002380,0.002294,0.002353,6429085,0.002412,0.002726,0.003318,0.004273,0.005921,False,False,False,False,True,False,False,False,True,True,False,False,True,True,True,False,True,True,True,True,29.756849,15.835088,32.766417,37.841621,-0.000314,-0.000352,0.000038,4,False,True,False,False,False,False,False,True,False,False,False,False,False
1502,0.407364,0.418191,0.403765,0.416116,21399374,0.399373,0.400923,0.410018,0.425475,0.429452,False,False,False,False,True,False,False,False,True,True,False,False,True,True,True,False,True,True,True,True,21.765687,20.224820,24.856122,54.389283,-0.001550,-0.007008,0.005458,6,True,False,False,False,False,False,False,False,False,True,False,False,False


51

In [80]:
dummies_input = dummies_recent.dropna()
print(len(dummies_input))
len(dummies_input.columns)


347


52

In [81]:
def data_clean(dados:pd.DataFrame, target_list:list, data_return:str):
    # Removing NA
    dados_treat = dados.dropna()

    # Substituindo valores infinitos por NaN
    dados_treat.replace([np.inf, -np.inf], np.nan, inplace=True)

    # Removendo linhas com valores NaN
    dados_treat.dropna(inplace=True)

    # Removing cols that won't be used in the model
    # removing_cols = ['Symbol', 'Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits']
    # removing_cols = ['Date']
    # removing_cols = ['Date', 'Symbol']
    removing_cols = ['Date', 'Symbol', 'Dividends', 'Stock Splits']

    # Define the target in a list of target (for futher iteration)
    dados_y = dados_treat[target_list]

    # Removing target from base to avoid data leakage
    dados_x = dados_treat.drop(dados_treat[target_list], axis=1)
    dados_x = dados_x.drop(dados_x[removing_cols], axis=1)

    if data_return == 'Y':
        return dados_y
    else:
        return dados_x

In [77]:
dados_x_all = data_clean(dados, remove_target_list, 'X')
dados_x_all_dummies = pd.get_dummies(dados_x_all)
len(dados_x_all_dummies.columns)

C:\Users\mathe\AppData\Local\Temp\ipykernel_16692\761462767.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dados_treat.replace([np.inf, -np.inf], np.nan, inplace=True)
C:\Users\mathe\AppData\Local\Temp\ipykernel_16692\761462767.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dados_treat.dropna(inplace=True)


52

In [84]:
# Encontrar colunas que estão no treinamento mas não na validação
missing_cols = set(dados_x_all_dummies.columns) - set(dummies_input.columns)

print(missing_cols)
# # Adicionar as colunas faltantes no conjunto de validação, preenchidas com zeros
for col in missing_cols:
    dummies_input[col] = False

# Reordenar as colunas no conjunto de validação para corresponder ao conjunto de treinamento
valid_dummies = dummies_input[dados_x_all_dummies.columns]

print("\nDados de validação alinhados com treinamento:")
display(valid_dummies.head(3))

set()

Dados de validação alinhados com treinamento:


,Open,High,Low,Close,Volume,ema_12,ema_26,ema_50,ema_100,ema_200,ema_12_above_ema_26,ema_12_above_ema_50,ema_12_above_ema_100,ema_12_above_ema_200,ema_26_above_ema_12,ema_26_above_ema_50,ema_26_above_ema_100,ema_26_above_ema_200,ema_50_above_ema_12,ema_50_above_ema_26,ema_50_above_ema_100,ema_50_above_ema_200,ema_100_above_ema_12,ema_100_above_ema_26,ema_100_above_ema_50,ema_100_above_ema_200,ema_200_above_ema_12,ema_200_above_ema_26,ema_200_above_ema_50,ema_200_above_ema_100,vl_adx,vl_dmp,vl_dmn,rsi,vl_macd,vl_macd_signal,vl_macd_delta,qt_days_macd_delta_positive,nm_adx_trend_1. Absent or Weak Trend,nm_adx_trend_2. Strong Trend,nm_adx_trend_3. Very Strong Trend,nm_adx_trend_4. Extremely Strong Trend,nm_rsi_trend_rsi_0_to_10,nm_rsi_trend_rsi_11_to_20,nm_rsi_trend_rsi_21_to_30,nm_rsi_trend_rsi_31_to_40,nm_rsi_trend_rsi_41_to_50,nm_rsi_trend_rsi_51_to_60,nm_rsi_trend_rsi_61_to_70,nm_rsi_trend_rsi_71_to_80,nm_rsi_trend_rsi_81_to_90,nm_rsi_trend_rsi_91_to_100
205,0.002310,0.002380,0.002294,0.002353,6429085,0.002412,0.002726,0.003318,0.004273,0.005921,False,False,False,False,True,False,False,False,True,True,False,False,True,True,True,False,True,True,True,True,29.756849,15.835088,32.766417,37.841621,-0.000314,-0.000352,0.000038,4,False,True,False,False,False,False,False,True,False,False,False,False,False,False
1502,0.407364,0.418191,0.403765,0.416116,21399374,0.399373,0.400923,0.410018,0.425475,0.429452,False,False,False,False,True,False,False,False,True,True,False,False,True,True,True,False,True,True,True,True,21.765687,20.224820,24.856122,54.389283,-0.001550,-0.007008,0.005458,6,True,False,False,False,False,False,False,False,False,True,False,False,False,False
1916,0.506377,0.512832,0.503943,0.505551,258097,0.510665,0.530978,0.560965,0.598849,0.613049,False,False,False,False,True,False,False,False,True,True,False,False,True,True,True,False,True,True,True,True,30.306626,16.874467,20.266179,32.286516,-0.020313,-0.023047,0.002734,5,False,True,False,False,False,False,False,True,False,False,False,False,False,False


In [87]:
#Fazendo a previsão das probabilidades
proba = clf.predict_proba(valid_dummies)

# Probabilidade de ser o target:
proba_target = proba[:,1] # array
# pro
# proba_target

c:\Users\mathe\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


array([0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 7.94439990e-028,
       0.00000000e+000, 1.58816942e-011, 0.00000000e+000, 0.00000000e+000,
       9.87411040e-050, 0.00000000e+000, 7.17048805e-039, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       9.88386645e-194, 0.00000000e+000, 0.00000000e+000, 1.98862951e-004,
       0.00000000e+000, 1.86341105e-001, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0

In [94]:
valid_dummies.head(3)

,Open,High,Low,Close,Volume,ema_12,ema_26,ema_50,ema_100,ema_200,ema_12_above_ema_26,ema_12_above_ema_50,ema_12_above_ema_100,ema_12_above_ema_200,ema_26_above_ema_12,ema_26_above_ema_50,ema_26_above_ema_100,ema_26_above_ema_200,ema_50_above_ema_12,ema_50_above_ema_26,ema_50_above_ema_100,ema_50_above_ema_200,ema_100_above_ema_12,ema_100_above_ema_26,ema_100_above_ema_50,ema_100_above_ema_200,ema_200_above_ema_12,ema_200_above_ema_26,ema_200_above_ema_50,ema_200_above_ema_100,vl_adx,vl_dmp,vl_dmn,rsi,vl_macd,vl_macd_signal,vl_macd_delta,qt_days_macd_delta_positive,nm_adx_trend_1. Absent or Weak Trend,nm_adx_trend_2. Strong Trend,nm_adx_trend_3. Very Strong Trend,nm_adx_trend_4. Extremely Strong Trend,nm_rsi_trend_rsi_0_to_10,nm_rsi_trend_rsi_11_to_20,nm_rsi_trend_rsi_21_to_30,nm_rsi_trend_rsi_31_to_40,nm_rsi_trend_rsi_41_to_50,nm_rsi_trend_rsi_51_to_60,nm_rsi_trend_rsi_61_to_70,nm_rsi_trend_rsi_71_to_80,nm_rsi_trend_rsi_81_to_90,nm_rsi_trend_rsi_91_to_100
205,0.002310,0.002380,0.002294,0.002353,6429085,0.002412,0.002726,0.003318,0.004273,0.005921,False,False,False,False,True,False,False,False,True,True,False,False,True,True,True,False,True,True,True,True,29.756849,15.835088,32.766417,37.841621,-0.000314,-0.000352,0.000038,4,False,True,False,False,False,False,False,True,False,False,False,False,False,False
1502,0.407364,0.418191,0.403765,0.416116,21399374,0.399373,0.400923,0.410018,0.425475,0.429452,False,False,False,False,True,False,False,False,True,True,False,False,True,True,True,False,True,True,True,True,21.765687,20.224820,24.856122,54.389283,-0.001550,-0.007008,0.005458,6,True,False,False,False,False,False,False,False,False,True,False,False,False,False
1916,0.506377,0.512832,0.503943,0.505551,258097,0.510665,0.530978,0.560965,0.598849,0.613049,False,False,False,False,True,False,False,False,True,True,False,False,True,True,True,False,True,True,True,True,30.306626,16.874467,20.266179,32.286516,-0.020313,-0.023047,0.002734,5,False,True,False,False,False,False,False,True,False,False,False,False,False,False


In [102]:
output = valid_dummies
output['proba_target'] = proba_target

C:\Users\mathe\AppData\Local\Temp\ipykernel_16692\87065134.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  output['proba_target'] = proba_target


In [105]:
output = output[['proba_target']]
output

,proba_target
205,0.0
1502,0.0
1916,0.0
3297,0.0
5399,0.0
...,...
880908,0.0
883294,0.0
884653,0.0
888029,0.0


In [111]:
last_date_join = last_date

In [112]:
last_date_join = last_date_join[['Symbol', 'Date', 'Close']]

In [113]:
# Fazendo o join dos DataFrames pelo índice
result = result = pd.merge(last_date_join, output, left_index=True, right_index=True)
result.head(2)

,Symbol,Date,Close,proba_target
205,1CAT-USD,2024-07-12 21:00:00-03:00,0.002353,0.0
1502,1INCH-USD,2024-07-12 21:00:00-03:00,0.416116,0.0


In [115]:
result.sort_values(by='proba_target', ascending=False)

,Symbol,Date,Close,proba_target
838159,XCN-USD,2024-07-12 21:00:00-03:00,0.000586,0.999769
654265,SCA-USD,2024-07-12 21:00:00-03:00,0.026332,0.999741
668079,SHRAP-USD,2024-07-12 21:00:00-03:00,15.754227,0.876286
780967,USDE-USD,2024-07-12 21:00:00-03:00,0.900394,0.837600
764066,UFO-USD,2024-07-12 21:00:00-03:00,0.000066,0.742967
...,...,...,...,...
561598,ORN-USD,2024-07-12 21:00:00-03:00,1.533827,0.000000
560139,ORDI-USD,2024-07-12 21:00:00-03:00,32.586960,0.000000
559709,ORCA-USD,2024-07-12 21:00:00-03:00,1.438488,0.000000
558640,ORBS-USD,2024-07-12 21:00:00-03:00,0.021870,0.000000


In [ ]:
### outras funções

In [ ]:
# from datetime import datetime
# from pathlib import Path
# import pyarrow.parquet as pq
# import pyarrow as pa

import json
import requests as re
import yfinance as yf
import pandas as pd


def get_kucoin_symbols():
    resp = re.get('https://api.kucoin.com/api/v2/symbols')
    ticker_list = json.loads(resp.content)
    symbols_list = [ticker['symbol'] for ticker in ticker_list['data'] if str(ticker['symbol'][-4:]) == 'USDT']
    return symbols_list

def fetch_crypto_data(symbols, start_date):
    df_list = []

    for count, symbol in enumerate(symbols):
        print(f'Processing {symbol} ({count + 1} of {len(symbols)})')

        # Get historical data using yfinance for the given symbol since start_date
        crypto_data = yf.Ticker(symbol).history(start=start_date)

        # Add a 'symbol' column to the DataFrame to identify the cryptocurrency
        crypto_data['Symbol'] = symbol

        # Append the DataFrame to the list if it's not empty
        if not crypto_data.empty:
            df_list.append(crypto_data)

    # Concatenate the list of DataFrames into a single DataFrame
    if df_list:
        return pd.concat(df_list)
    else:
        return None

kucoin_symbols = get_kucoin_symbols()

data_hoje = pd.Timestamp.today()

# Formatando a data no formato 'YYYY-MM-DD'
data_hoje_formatada = data_hoje.strftime('%Y-%m-%d')

crypto_symbols = [symbol[:-1] for symbol in kucoin_symbols]

crypto_today = fetch_crypto_data(crypto_symbols, data_hoje_formatada)

In [ ]:

from indicators_util import add_indicators

# Add indicators to the DataFrame
indicators_dataframe = add_indicators(crypto_today)